In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import os, shutil
import zipfile

In [ ]:
zip_ref = zipfile.ZipFile('../input/dogs-vs-cats/train.zip', 'r')
zip_ref.extractall('train')
zip_ref.close()

zip_ref = zipfile.ZipFile('../input/dogs-vs-cats/test1.zip', 'r')
zip_ref.extractall('test')
zip_ref.close()

In [ ]:
train_full_dir = './train/train/'
test_full_dir = './test/test1/'

create_if_not = ['training/dogs/','training/cats/','testing/']
for i in create_if_not:
    if os.path.isdir(i) == 0:
        os.makedirs(i)

In [ ]:
train_filenames = os.listdir(train_full_dir)
categories = []
for tr_filename in train_filenames:
    category = tr_filename.split('.')[0]
    if category == 'dog':
        shutil.move(train_full_dir+ tr_filename, 'training/dogs/'+ tr_filename)
    else:
        shutil.move(train_full_dir+ tr_filename, 'training/cats/'+ tr_filename)

test_filenames = os.listdir(test_full_dir)
for t_filename in test_filenames:
        shutil.move(test_full_dir+ t_filename, 'testing/'+ t_filename)
        
shutil.rmtree('./train')
shutil.rmtree('./test')

In [ ]:
print("Training Dog images: ",len(os.listdir('./training/dogs')))
print("Training Cat images: ",len(os.listdir('./training/cats')))
print("Testing images: ",len(os.listdir('./testing')))

In [ ]:
image_size = (180,180)
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    './training',
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    './training',
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

In [ ]:
class_names = train_ds.class_names

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
from tensorflow.keras.models import Sequential
num_classes = 2

model = Sequential([
    layers.experimental.preprocessing.Rescaling(1./255, input_shape=(180, 180, 3)),
    layers.experimental.preprocessing.RandomRotation(0.2),
    layers.experimental.preprocessing.RandomZoom(0.2),

    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes)
])
model.compile(optimizer='adam',
              loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()

In [ ]:
epochs=50
callbacks = [keras.callbacks.ModelCheckpoint("./ckpt/save_at_{epoch}.h5")]
history = model.fit(train_ds, validation_data=val_ds,epochs=epochs, callbacks=callbacks)

In [ ]:
model.save('catsvsdogs-simple.h5')

In [ ]:
def display_training_curves(training, validation, title, subplot):
    ax = plt.subplot(subplot)
    ax.plot(training)
    ax.plot(validation)
    ax.set_title('model '+ title)
    ax.set_ylabel(title)
    ax.set_xlabel('epoch')
    ax.legend(['training', 'validation'])

plt.subplots(figsize=(10,10))
plt.tight_layout()
display_training_curves(history.history['accuracy'], history.history['val_accuracy'], 'accuracy', 211)
display_training_curves(history.history['loss'], history.history['val_loss'], 'loss', 212)

In [ ]:
import random
test_filenames = os.listdir('./testing')
rand = random.choice(test_filenames)
image_path = './testing/' + rand
img = keras.preprocessing.image.load_img(image_path, target_size=(180, 180))
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)].upper(), 100 * np.max(score))
)


from PIL import Image
Image.open(image_path)  